In [91]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 
import os
from matplotlib.font_manager import FontProperties
sns.set(font=['sans-serif'])
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})
# scikit-learn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import datasets, linear_model, preprocessing, neighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold,RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
import xgboost as xgb

In [92]:
# load data
# df= pd.read_csv('taipei_dataset_no_encoding.csv', engine='python')
df= pd.read_csv('taipei_dataset_addB.csv', engine='python')

In [ ]:
# load data
df= pd.read_csv('new_taipei_dataset_no_encoding.csv', engine='python')

In [ ]:
# load data
df= pd.read_csv('taichung_dataset_no_encoding.csv', engine='python')

In [ ]:
# load data
df= pd.read_csv('allcity_dataset_no_encoding.csv', engine='python')

In [ ]:
# load data
df= pd.read_csv('allcity_dataset_no_encoding.csv', engine='python')

In [ ]:
# load data
df= pd.read_csv('allcity_dataset_no_encoding.csv', engine='python')

In [93]:
df_main=df.copy()
df_main=df_main.drop(['Unnamed: 0'],axis=1)
# df_main=df_main.drop(df_main[df_main['移轉層次']=='全'].index)
# df_main=df_main.reset_index(drop=True)

In [94]:
new_all_floor = {"總樓層數": 
             {"一層" : 1, 
              "二層" : 2, 
              "三層" : 3, 
              "四層" : 4, 
              "五層" : 5, 
              "六層" : 6, 
              "七層" : 7, 
              "八層" : 8, 
              "九層" : 9, 
              "十層" : 10, 
              "十一層" : 11, 
              "十二層" : 12, 
              "十三層" : 13, 
              "十四層" : 14, 
              "十五層" : 15, 
              "十六層" : 16, 
              "十七層" : 17, 
              "十八層" : 18, 
              "十九層" : 19, 
              "二十層" : 20,
              "二十一層" : 21, 
              "二十二層" : 22, 
              "二十三層" : 23, 
              "二十四層" : 24, 
              "二十五層" : 25, 
              "二十六層" : 26, 
              "二十七層" : 27, 
              "二十八層" : 28, 
              "二十九層" : 29, 
              "三十層" : 30,
              "三十一層" : 31,
              "三十二層" : 32,
              "三十三層" : 33,
              "三十四層" : 34,
              "三十五層" : 35,
              "三十六層" : 36,
              "三十七層" : 37,
              "三十八層" : 38,
              "三十九層" : 39,
              "四十層" : 40,
              "四十一層" : 41,
              "四十二層" : 42,
              "四十三層" : 43,
              "四十六層" : 46,
              "六十八層" : 68
             }
            }

In [95]:
df_floor_all = df_main.replace(new_all_floor)
df_floor_all["總樓層數"]=df_floor_all["總樓層數"].astype('int64')

In [96]:
df_floor_all=df_floor_all.drop(df_floor_all[(df_floor_all['建物型態']=='透天厝') & (df_floor_all['總樓層數']>8)].index)
df_floor_all
# df_floor_all=df_floor_all.reset_index(drop=True)

,鄉鎮市區,交易標的,都市土地使用分區,交易年,移轉層次,總樓層數,建物型態,主要用途,主要建材,有無管理組織,建物移轉總面積坪,每坪價格,土地,建物,車位,屋齡,房間數
0,文山區,房地(土地+建物),商,105,七層,33,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,有,34.932700,64.695829,1,1,0,1,2
1,文山區,房地(土地+建物),住,105,二層,10,套房(1房1廳1衛),住家用,鋼筋混凝土造,有,12.405525,48.365547,1,1,0,8,1
2,文山區,房地(土地+建物)+車位,住,105,九層,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,66.271700,44.061040,3,1,1,5,3
3,文山區,房地(土地+建物)+車位,住,105,四層,5,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,有,69.039575,50.695561,1,1,1,7,4
4,萬華區,房地(土地+建物),住,105,九層,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,24.293775,50.218626,1,1,0,34,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142573,文山區,房地(土地+建物)+車位,住,110,五層,27,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,無,58.430900,60.926667,1,1,1,0,3
142574,文山區,房地(土地+建物)+車位,住,110,十七,27,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,無,68.274250,66.935924,1,1,1,0,3
142575,南港區,房地(土地+建物)+車位,住,110,五層,11,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,無,23.232000,72.098829,1,1,1,0,1
142576,南港區,房地(土地+建物)+車位,住,110,三層,11,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,無,32.924100,77.815339,1,1,1,0,2


In [97]:
new_floor = {"移轉層次": 
             {"一層" : 1, 
              "二層" : 2, 
              "三層" : 3, 
              "四層" : 4, 
              "五層" : 5, 
              "六層" : 6, 
              "七層" : 7, 
              "八層" : 8, 
              "九層" : 9, 
              "十層" : 10, 
              "十一" : 11, 
              "十二" : 12, 
              "十三" : 13, 
              "十四" : 14, 
              "十五" : 15, 
              "十六" : 16, 
              "十七" : 17, 
              "十八" : 18, 
              "十九" : 19, 
              "二十" : 20,
              "三十" : 30
             }
            }

In [126]:
df_floornum = df_floor_all.replace(new_floor)
df_floornum["移轉層次"]=df_floornum["移轉層次"].astype('int64')
df_floornum["交易年"]=df_floornum["交易年"].astype('str')
# df_floornum.head(5)

In [102]:
df_oh=pd.get_dummies(df_floornum)
df_oh

,移轉層次,總樓層數,建物移轉總面積坪,每坪價格,土地,建物,車位,屋齡,房間數,鄉鎮市區_中山區,...,主要用途_國民住宅,主要用途_集合住宅,主要建材_加強磚造,主要建材_見其他登記事項,主要建材_鋼筋混凝土造,主要建材_鋼骨混凝土造,主要建材_鋼骨造,主要建材_鋼骨鋼筋混凝土造,有無管理組織_有,有無管理組織_無
0,7,33,34.932700,64.695829,1,1,0,1,2,0,...,0,0,0,0,0,0,0,1,1,0
1,2,10,12.405525,48.365547,1,1,0,8,1,0,...,0,0,0,0,1,0,0,0,1,0
2,9,12,66.271700,44.061040,3,1,1,5,3,0,...,0,0,0,0,1,0,0,0,1,0
3,4,5,69.039575,50.695561,1,1,1,7,4,0,...,0,0,0,0,1,0,0,0,1,0
4,9,12,24.293775,50.218626,1,1,0,34,2,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142573,5,27,58.430900,60.926667,1,1,1,0,3,0,...,0,0,0,0,0,0,0,1,0,1
142574,17,27,68.274250,66.935924,1,1,1,0,3,0,...,0,0,0,0,0,0,0,1,0,1
142575,5,11,23.232000,72.098829,1,1,1,0,1,0,...,0,0,0,0,1,0,0,0,0,1
142576,3,11,32.924100,77.815339,1,1,1,0,2,0,...,0,0,0,0,1,0,0,0,0,1


In [11]:
df_code=df_floornum.copy()
df_code["鄉鎮市區"] = pd.Categorical(df_code["鄉鎮市區"]).codes.astype('int64')
df_code["交易標的"] = pd.Categorical(df_code["交易標的"]).codes.astype('int64')
df_code["交易年"] = pd.Categorical(df_code["交易年"]).codes.astype('int64')
df_code["建物型態"] = pd.Categorical(df_code["建物型態"]).codes.astype('int64')
df_code["都市土地使用分區"] = pd.Categorical(df_code["都市土地使用分區"]).codes.astype('int64')
df_code["主要用途"] = pd.Categorical(df_code["主要用途"]).codes.astype('int64')
df_code["主要建材"] = pd.Categorical(df_code["主要建材"]).codes.astype('int64')
df_code["有無管理組織"] = pd.Categorical(df_code["有無管理組織"]).codes.astype('int64')
df_code["是否為預售屋"] = pd.Categorical(df_code["是否為預售屋"]).codes.astype('int64')
df_code.dtypes

鄉鎮市區          int64
交易標的          int64
都市土地使用分區      int64
交易年           int64
移轉層次          int64
總樓層數          int64
建物型態          int64
主要用途          int64
主要建材          int64
有無管理組織        int64
建物移轉總面積坪    float64
每坪價格        float64
土地            int64
建物            int64
車位            int64
屋齡            int64
是否為預售屋        int64
房間數           int64
dtype: object

In [134]:
df_code=df_floornum.copy()
df_code["鄉鎮市區"] = pd.Categorical(df_code["鄉鎮市區"])
df_code["交易標的"] = pd.Categorical(df_code["交易標的"])
df_code["交易年"] = pd.Categorical(df_code["交易年"])
df_code["建物型態"] = pd.Categorical(df_code["建物型態"])
df_code["都市土地使用分區"] = pd.Categorical(df_code["都市土地使用分區"])
df_code["主要用途"] = pd.Categorical(df_code["主要用途"])
df_code["主要建材"] = pd.Categorical(df_code["主要建材"])
df_code["有無管理組織"] = pd.Categorical(df_code["有無管理組織"])
# df_code["是否為預售屋"] = pd.Categorical(df_code["是否為預售屋"])
df_code

,鄉鎮市區,交易標的,都市土地使用分區,交易年,移轉層次,總樓層數,建物型態,主要用途,主要建材,有無管理組織,建物移轉總面積坪,每坪價格,土地,建物,車位,屋齡,房間數
0,文山區,房地(土地+建物),商,105,7,33,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,有,34.932700,64.695829,1,1,0,1,2
1,文山區,房地(土地+建物),住,105,2,10,套房(1房1廳1衛),住家用,鋼筋混凝土造,有,12.405525,48.365547,1,1,0,8,1
2,文山區,房地(土地+建物)+車位,住,105,9,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,66.271700,44.061040,3,1,1,5,3
3,文山區,房地(土地+建物)+車位,住,105,4,5,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,有,69.039575,50.695561,1,1,1,7,4
4,萬華區,房地(土地+建物),住,105,9,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,24.293775,50.218626,1,1,0,34,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142573,文山區,房地(土地+建物)+車位,住,110,5,27,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,無,58.430900,60.926667,1,1,1,0,3
142574,文山區,房地(土地+建物)+車位,住,110,17,27,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,無,68.274250,66.935924,1,1,1,0,3
142575,南港區,房地(土地+建物)+車位,住,110,5,11,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,無,23.232000,72.098829,1,1,1,0,1
142576,南港區,房地(土地+建物)+車位,住,110,3,11,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,無,32.924100,77.815339,1,1,1,0,2


## Random Forest Feature Selection

In [103]:
from sklearn import feature_selection
selector = feature_selection.SelectPercentile(feature_selection.f_regression, percentile=30)
selector.fit(X, y)
selector.get_support(True)

array([ 0,  1,  6,  7,  8,  9, 12, 16, 17, 18, 19, 23, 25, 36, 45, 46, 55,
       56, 59], dtype=int64)

In [104]:
from sklearn.feature_selection import RFECV, RFE

estimator=RandomForestRegressor(n_estimators=50, n_jobs=8)
rfe=RFE(estimator)
rfe.fit(X, y)
rfe.ranking_

array([ 1,  1,  1,  1,  4,  1,  1,  1,  1,  1,  1,  1,  1, 10,  1, 12,  1,
        1,  1,  1, 30,  1,  1,  1,  9,  1, 21,  1,  1,  1,  1,  1,  5,  7,
        3,  1,  1, 14, 27, 28, 32, 29, 31, 19, 17, 11,  1, 15,  6, 24, 18,
        8, 16, 23, 20, 22,  1, 26, 25, 13,  2,  1])

In [110]:
X_del=pd.DataFrame({'ranking':rfe.ranking_,'features':X.columns}).sort_values(by=['ranking'])
n=[]
s=0
for i in X_del['features']:
    if s<30:
        n.append(i)
        s+=1
    else:
        break
X[n].columns

Index(['移轉層次', '主要建材_鋼筋混凝土造', '建物型態_公寓(5樓含以下無電梯)', '交易年_110', '交易年_109',
       '交易年_105', '交易年_103', '交易年_102', '交易年_101', '都市土地使用分區_商', '都市土地使用分區_住',
       '交易標的_房地(土地+建物)+車位', '交易標的_房地(土地+建物)', '鄉鎮市區_萬華區', '鄉鎮市區_松山區',
       '鄉鎮市區_文山區', '交易年_104', '有無管理組織_無', '建物移轉總面積坪', '土地', '車位', '屋齡',
       '鄉鎮市區_大安區', '鄉鎮市區_中山區', '鄉鎮市區_中正區', '房間數', '鄉鎮市區_信義區', '鄉鎮市區_內湖區',
       '鄉鎮市區_北投區', '總樓層數'],
      dtype='object')

In [111]:
X=X[n]
y=df_code_mod['每坪價格']

## XGBRegressor

In [139]:
X=df_code.drop(columns=['每坪價格'])
y=df_code['每坪價格']


In [83]:
X=df_oh.drop(columns=['每坪價格'])
y=df_oh['每坪價格']

In [142]:
# 調參前
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)

# 建立 XGBRegressor 模型
xgbr=xgb.XGBRegressor(tree_method="gpu_hist",enable_categorical=True)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

# X_test = scaler.transform(X_test)
y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)
# score = xgb.score(X_test, y_test)
# importance=xgb.feature_importances_
print(' 訓練集MSE: %.3f, 測試集: %.3f' % (
mean_squared_error(y_train, y_train_pred),
mean_squared_error(y_test, y_test_pred)))
print(' 訓練集R2: %.3f, 測試集R2: %.3f' % (
r2_score(y_train, y_train_pred),
r2_score(y_test, y_test_pred)))

 訓練集MSE: 126.181, 測試集: 150.860
 訓練集R2: 0.703, 測試集R2: 0.643


In [ ]:
os.environ["PATH"] += os.pathsep + '../../pylab-Roger/BDSE_機器學習/實作/hands-on_part5/example/release/bin'
# Get a graph
graph = xgb.to_graphviz(xgbr, num_trees=1)
# Or get a matplotlib axis
ax = xgb.plot_tree(xgbr, num_trees=1)
# Get feature importances
xgbr.feature_importances_

In [ ]:
# grid 調參
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

params = {'max_depth': [3, 6, 10, 15],
              'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
              'subsample': np.arange(0.5, 1.0, 0.1),
              'colsample_bytree': np.arange(0.5, 1.0, 0.1),
              'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
              'n_estimators': [100, 250, 500, 750],
              }

xgbclf = xgb.XGBRegressor(tree_method='gpu_hist',enable_categorical=True)
clf = RandomizedSearchCV(estimator=xgbclf,
                             param_distributions=params,
                             n_iter=25,
                             n_jobs=4,
                             verbose=1)

clf.fit(X_train, y_train)

print(clf.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [ ]:
# summarize results
print("Best: %f using %s" % (rf_reg_tuned.best_score_, rf_reg_tuned.best_params_))
means = rf_reg_tuned.cv_results_[ 'mean_test_score' ]
stds = rf_reg_tuned.cv_results_[ 'std_test_score' ]
params = rf_reg_tuned.cv_results_[ 'params' ]
print(means,stds,params)

In [120]:
# 調參後
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)

xgbr=xgb.XGBRegressor(tree_method="gpu_hist",enable_categorical=True,n_jobs=8,
                      max_depth=10,
                      n_estimators=300,
                      alpha=10,
                      learning_rate=0.045,
                      colsample_bytree=0.8)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

# X_test = scaler.transform(X_test)
y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)
# score = xgb.score(X_test, y_test)
# importance=xgb.feature_importances_
print(' 訓練集MSE: %.3f, 測試集: %.3f' % (
mean_squared_error(y_train, y_train_pred),
mean_squared_error(y_test, y_test_pred)))
print(' 訓練集R2: %.3f, 測試集R2: %.3f' % (
r2_score(y_train, y_train_pred),
r2_score(y_test, y_test_pred)))

 訓練集MSE: 97.931, 測試集: 136.957
 訓練集R2: 0.766, 測試集R2: 0.671


In [ ]:
os.environ["PATH"] += os.pathsep + '../../pylab-Roger/BDSE_機器學習/實作/hands-on_part5/example/release/bin'

for i_tree, tree_in_forest in enumerate(model.estimators_):
    dot_data = tree.export_graphviz(tree_in_forest, out_file = None)
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render('./random_forest_plot/tree_' + str(i_tree)) 
    from IPython.display import display, Image
    Image(graph.render('output-graph.gv'))
    break

In [ ]:
kf =KFold(n_splits=5, shuffle=True, random_state=42)
sc=cross_val_score(model,X, y, cv= kf)

In [ ]:
sc.mean()

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
score = model.score(X_test, y_test)
importance=model.feature_importances_
print(' 訓練集MSE: %.3f, 測試集: %.3f' % (
mean_squared_error(y_train, y_train_pred),
mean_squared_error(y_test, y_test_pred)))
print(' 訓練集R2: %.3f, 測試集R2: %.3f' % (
r2_score(y_train, y_train_pred),
r2_score(y_test, y_test_pred)))

In [ ]:
# 調參後
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# 預設參數
model = RandomForestRegressor(n_jobs=8)
# model = RandomForestRegressor(n_jobs=8, max_depth=30, max_leaf_nodes=50, min_samples_leaf=20)
model.fit(X_train, y_train)

# X_test = scaler.transform(X_test)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
# score = model.score(X_test, y_test)
# importance=model.feature_importances_

print(' 訓練集MSE: %.3f, 測試集: %.3f' % (
mean_squared_error(y_train, y_train_pred),
mean_squared_error(y_test, y_test_pred)))
print(' 訓練集R2: %.3f, 測試集R2: %.3f' % (
r2_score(y_train, y_train_pred),
r2_score(y_test, y_test_pred)))